In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

df = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [3]:
spark.version

'3.3.2'

In [14]:
from pyspark.sql.types import *

fhv_schema = StructType([
    StructField('dispatching_base_num', StringType(), True), 
    StructField('pickup_datetime', TimestampType(), True), 
    StructField('dropOff_datetime', TimestampType(), True), 
    StructField('PUlocationID', IntegerType(), True), 
    StructField('DOlocationID', IntegerType(), True), 
    StructField('SR_Flag', StringType(), True), 
    StructField('Affiliated_base_number', StringType(), True)
])

In [16]:
fhv = spark.read \
    .option("header", "true") \
    .option("compression", "gzip") \
    .schema(fhv_schema) \
    .csv('fhv_tripdata_2019-10.csv.gz')

In [7]:
fhv \
.repartition(6) \
.write \
.mode('overwrite') \
.parquet('fhv')

In [17]:
from pyspark.sql.functions import col

fhv \
.filter((col('pickup_datetime') >= '2019-10-15') & (col('pickup_datetime') < '2019-10-16')) \
.count()

62610

In [23]:
fhv \
.withColumn('length', (col('dropOff_datetime').cast(LongType()) - col('pickup_datetime').cast(LongType()))) \
.withColumn('length_in_hours', col('length') / 3600) \
.orderBy(col('length_in_hours').desc()) \
.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|    length|   length_in_hours|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+------------------+
|              B02832|2019-10-11 18:00:00|2091-10-11 18:30:00|         264|         264|   null|                B02832|2272149000|          631152.5|
|              B02832|2019-10-28 09:00:00|2091-10-28 09:30:00|         264|         264|   null|                B02832|2272149000|          631152.5|
|              B02416|2019-10-31 23:46:33|2029-11-01 00:13:00|        null|        null|   null|                B02416| 315620787| 87672.44083333333|
|     B00746         |2019-10-01 21:43:42|2027-10-01 21:45:23|         159|         264|   null|    

In [24]:
df.createTempView('zones')

In [26]:
spark.sql('select * from zones').show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [29]:
fhv.createOrReplaceTempView('fhv')

In [35]:
spark.sql("""
select Zone, count(*) as trips
from fhv f
inner join zones z
on f.PUlocationID = z.LocationID
group by Zone
order by trips asc
""").show(10)

+--------------------+-----+
|                Zone|trips|
+--------------------+-----+
|         Jamaica Bay|    1|
|Governor's Island...|    2|
| Green-Wood Cemetery|    5|
|       Broad Channel|    8|
|     Highbridge Park|   14|
|        Battery Park|   15|
|Saint Michaels Ce...|   23|
|Breezy Point/Fort...|   25|
|Marine Park/Floyd...|   26|
|        Astoria Park|   29|
+--------------------+-----+
only showing top 10 rows

